In [1]:
%matplotlib inline

import os

import mne
import numpy as np

## Short exercise for research assistant position

The research assistant position will include tasks to prepare (i.e., preprocess) neuroimaging data such as EEG and fMRI data mostly using Python, and to apply data analysis techniques to these data. 

In this short exercise I ask you to write code to import and manipulate EEG data. Before you start, you need to install Python, Ipython Notebook (if not available on your computer) and MNE. 

## Timeseries data in MNE - the `Raw` class

The most basic form of electrophysiological data is timeseries data: a continuous set of voltage values recorded over time for each [channel](https://mne.tools/stable/documentation/glossary.html#term-channels).

Timeseries data in MNE is stored in [`mne.io.Raw`](https://mne.tools/stable/generated/mne.io.Raw.html) and [`mne.io.RawArray`](https://mne.tools/stable/generated/mne.io.RawArray.html) objects.

`Raw` objects can be created through loading data from the disk via one of the various [`mne.io.read_raw_xxx()`](https://mne.tools/stable/api/reading_raw_data.html) functions.

`RawArray` objects can be created from data [arrays](https://numpy.org/doc/stable/reference/arrays.html) directly.

### Part 1 - Reading timeseries data from disk

To familiarise ourselves with `Raw` objects, we will start by loading [MNE's sample dataset](https://mne.tools/stable/documentation/datasets.html#sample).

In [2]:
# Filepath to MNE's sample dataset on disk
sample_data_folder = mne.datasets.sample.data_path()

# Load sample data from disk
raw = mne.io.read_raw_fif(
    os.path.join(sample_data_folder, "MEG", "sample", "sample_audvis_raw.fif")
)

Opening raw data file C:\Users\Jishan\mne_data\MNE-sample-data\MEG\sample\sample_audvis_raw.fif...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.


C:\Users\Jishan\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


`Raw` objects contain:
- the timeseries data
- the metadata, stored under the `info` attribute

In [3]:
# Show information about the data object
raw.info

Measurement date,"December 03, 2002 19:01:10 GMT"
Experimenter,MEG
Participant,Unknown
Digitized points,146 points
Good channels,"203 Gradiometers, 102 Magnetometers, 9 Stimulus, 59 EEG, 1 EOG"
Bad channels,"MEG 2443, EEG 053"
EOG channels,EOG 061
ECG channels,Not available
Sampling frequency,600.61 Hz
Highpass,0.10 Hz
Lowpass,172.18 Hz


For example, here you can see that the data we have loaded contains a mixture of MEG data (gradiometers and magnetometers) and EEG data sampled at 600 Hz, as well as the timings of stimulus presentation and subject behaviour during the recording ([stimulus channels](https://mne.tools/stable/documentation/glossary.html#term-stim-channel)).

The data itself can be accessed via the [`get_data()`](https://mne.tools/stable/generated/mne.io.Raw.html#mne.io.Raw.get_data) method, which returns an array of shape `(channels, times)`.

In [4]:
# Get data as an array
data = raw.get_data()
print(f"Data has shape: {data.shape} (channels, times)")

Data has shape: (376, 166800) (channels, times)


`Raw` objects have various methods for working with timeseries data, such as:
- isolating specific channels - [`pick()`](https://mne.tools/stable/generated/mne.io.Raw.html#mne.io.Raw.pick), [`drop_channels()`](https://mne.tools/stable/generated/mne.io.Raw.html#mne.io.Raw.drop_channels)
- isolating specific windows of time - [`crop()`](https://mne.tools/stable/generated/mne.io.Raw.html#mne.io.Raw.crop)
- spectral filtering of the data - [`filter()`](https://mne.tools/stable/generated/mne.io.Raw.html#mne.io.Raw.filter), [`notch_filter()`](https://mne.tools/stable/generated/mne.io.Raw.html#mne.io.Raw.notch_filter)
- plotting the data - [`plot()`](https://mne.tools/stable/generated/mne.io.Raw.html#mne.io.Raw.plot)
- computing the power spectra of the data - [`compute_psd()`](https://mne.tools/stable/generated/mne.io.Raw.html#mne.io.Raw.compute_psd)

We will explore some of these methods below, and others in later notebooks.

**Exercises - Manipulating `Raw` objects**

We will start by selecting only a subset of channels to store in our `Raw` object, using the [`pick()`](https://mne.tools/stable/generated/mne.io.Raw.html#mne.io.Raw.pick) method.

`pick()` accepts channel names, channel types, or channel indices as input, and retains only those channels that match this criteria.

Below, we select only the MEG channels.

*Hint:* Generally in MNE, methods will modify the object in-place to save memory. Because we want to play around with the data without modifying the original object, we will first make a copy of the `Raw` object using the [`copy()`](https://mne.tools/stable/generated/mne.io.Raw.html#mne.io.Raw.copy) method.

In [5]:
# Create a copy of the data so that we can manipulate it
raw_copy = raw.copy()

# Select the MEG channels only
raw_copy.pick("meg")

# Verify that we have only MEG channels
print(raw_copy.get_data().shape)
raw_copy.info

(306, 166800)


Measurement date,"December 03, 2002 19:01:10 GMT"
Experimenter,MEG
Participant,Unknown
Digitized points,146 points
Good channels,"203 Gradiometers, 102 Magnetometers"
Bad channels,MEG 2443
EOG channels,Not available
ECG channels,Not available
Sampling frequency,600.61 Hz
Highpass,0.10 Hz
Lowpass,172.18 Hz


As you can see, the new `Raw` object now has only 306 channels, corresponding to the 203 Gradiometers and 102 Magnetometers (as well as 1 'bad' channel where the data was not properly recorded).

**Exercise:** Select only the EEG channels from the original `Raw` object, and verify that only these channels remain.

In [6]:
# Remember to copy the original data!
raw_copy = raw.copy()

## CODE GOES HERE
# Select the EEG channels only
raw_copy.pick("eeg")

# Verify that we have only EEG channels
print(raw_copy.get_data().shape)
print(raw_copy.info['nchan'])
raw_copy.info

Removing projector <Projection | PCA-v1, active : False, n_channels : 102>
Removing projector <Projection | PCA-v2, active : False, n_channels : 102>
Removing projector <Projection | PCA-v3, active : False, n_channels : 102>
(60, 166800)
60


Measurement date,"December 03, 2002 19:01:10 GMT"
Experimenter,MEG
Participant,Unknown
Digitized points,146 points
Good channels,59 EEG
Bad channels,EEG 053
EOG channels,Not available
ECG channels,Not available
Sampling frequency,600.61 Hz
Highpass,0.10 Hz
Lowpass,172.18 Hz


In this part, I choose EEG channels. In the output, it is clearly stated that there are 60 channels for EEG. There are 59 EEG channels that are good, and 1 EEG channel is bad, which is "EEG 053".

**Exercise:** Now select the EEG and MEG channels simultaneously, and verify that only these channels remain.

In [7]:
# Remember to copy the original data
raw_copy = raw.copy()

## CODE GOES HERE
# Select EEG and MEG channels
raw_copy.pick(["eeg", "meg"])

# Verify that we have only EEG and MEG channels
print(raw_copy.get_data().shape)
b = raw_copy.info['nchan']
raw_copy.info

(366, 166800)


Measurement date,"December 03, 2002 19:01:10 GMT"
Experimenter,MEG
Participant,Unknown
Digitized points,146 points
Good channels,"203 Gradiometers, 102 Magnetometers, 59 EEG"
Bad channels,"MEG 2443, EEG 053"
EOG channels,Not available
ECG channels,Not available
Sampling frequency,600.61 Hz
Highpass,0.10 Hz
Lowpass,172.18 Hz


In this part, I choose EEG and MEG channels simultaneously. In the output, it is clearly stated that there are 366 channels in the EEG and MEG. There are 59 EEG channels that are good, and 1 EEG channel is bad, which is "EEG053". Moreover, in the MEG portion, "MEG 2443" is under the bad channel. 


**Exercise:** Select the EEG and stimulus channels simultaneously, and verify that only these channels remain.

In [8]:
raw_copy = raw.copy()

## CODE GOES HERE
# Select EEG and stimulus channels
raw_copy.pick(["eeg", "stim"])

# Verify that we have only EEG and stimulus channels
print(raw_copy.get_data().shape)
a = raw_copy.info['nchan']
raw_copy.info

Removing projector <Projection | PCA-v1, active : False, n_channels : 102>
Removing projector <Projection | PCA-v2, active : False, n_channels : 102>
Removing projector <Projection | PCA-v3, active : False, n_channels : 102>
(69, 166800)


Measurement date,"December 03, 2002 19:01:10 GMT"
Experimenter,MEG
Participant,Unknown
Digitized points,146 points
Good channels,"9 Stimulus, 59 EEG"
Bad channels,EEG 053
EOG channels,Not available
ECG channels,Not available
Sampling frequency,600.61 Hz
Highpass,0.10 Hz
Lowpass,172.18 Hz


In this part, I choose EEG and Stimulus channels simultaneously. In the output, it is clearly stated that there are 69 channels in the EEG and Stimulus. There are 59 EEG channels that are good, and 1 EEG channel is bad, which is "EEG053". Moreover, there are no bad channels in the Stimulus.

**Exercise:** Select three channels of your choice by specifying their names, and verify that only these channels remain.

*Hint:* channel names are stored in the `ch_names` attribute of the `Raw` object.

In [9]:
raw_copy = raw.copy()

## CODE GOES HERE

# Select three channels of your choice by specifying their names
ch_names = ["eog", "stim", "eeg"] #I also checked 'seeg', 'hbo', 'hbr', but did not find out anything.
raw_copy.pick(ch_names)

# Verify that we have only the selected channels
print(raw_copy.get_data().shape)
d = raw_copy.info['nchan']
raw_copy.info

Removing projector <Projection | PCA-v1, active : False, n_channels : 102>
Removing projector <Projection | PCA-v2, active : False, n_channels : 102>
Removing projector <Projection | PCA-v3, active : False, n_channels : 102>
(70, 166800)


Measurement date,"December 03, 2002 19:01:10 GMT"
Experimenter,MEG
Participant,Unknown
Digitized points,146 points
Good channels,"9 Stimulus, 59 EEG, 1 EOG"
Bad channels,EEG 053
EOG channels,EOG 061
ECG channels,Not available
Sampling frequency,600.61 Hz
Highpass,0.10 Hz
Lowpass,172.18 Hz


In [10]:
raw_copy.ch_names

['STI 001',
 'STI 002',
 'STI 003',
 'STI 004',
 'STI 005',
 'STI 006',
 'STI 014',
 'STI 015',
 'STI 016',
 'EEG 001',
 'EEG 002',
 'EEG 003',
 'EEG 004',
 'EEG 005',
 'EEG 006',
 'EEG 007',
 'EEG 008',
 'EEG 009',
 'EEG 010',
 'EEG 011',
 'EEG 012',
 'EEG 013',
 'EEG 014',
 'EEG 015',
 'EEG 016',
 'EEG 017',
 'EEG 018',
 'EEG 019',
 'EEG 020',
 'EEG 021',
 'EEG 022',
 'EEG 023',
 'EEG 024',
 'EEG 025',
 'EEG 026',
 'EEG 027',
 'EEG 028',
 'EEG 029',
 'EEG 030',
 'EEG 031',
 'EEG 032',
 'EEG 033',
 'EEG 034',
 'EEG 035',
 'EEG 036',
 'EEG 037',
 'EEG 038',
 'EEG 039',
 'EEG 040',
 'EEG 041',
 'EEG 042',
 'EEG 043',
 'EEG 044',
 'EEG 045',
 'EEG 046',
 'EEG 047',
 'EEG 048',
 'EEG 049',
 'EEG 050',
 'EEG 051',
 'EEG 052',
 'EEG 053',
 'EEG 054',
 'EEG 055',
 'EEG 056',
 'EEG 057',
 'EEG 058',
 'EEG 059',
 'EEG 060',
 'EOG 061']

In [11]:
raw_copy = raw.copy()

## CODE GOES HERE
df = raw_copy.ch_names
#print(df)
# Select three channels of your choice by specifying their names
ch_names = ["EEG 001", "STI 001", "EOG 061"]
raw_copy.pick(ch_names)

# Verify that we have only the selected channels
print(raw_copy.get_data().shape)
d = raw_copy.info['nchan']
raw_copy.info

Removing projector <Projection | PCA-v1, active : False, n_channels : 102>
Removing projector <Projection | PCA-v2, active : False, n_channels : 102>
Removing projector <Projection | PCA-v3, active : False, n_channels : 102>
(3, 166800)


Measurement date,"December 03, 2002 19:01:10 GMT"
Experimenter,MEG
Participant,Unknown
Digitized points,146 points
Good channels,"1 EEG, 1 Stimulus, 1 EOG"
Bad channels,None
EOG channels,EOG 061
ECG channels,Not available
Sampling frequency,600.61 Hz
Highpass,0.10 Hz
Lowpass,172.18 Hz


In this part, I choose EOG, EEG, and Stimulus channels simultaneously. There are 59 EEG channels, 1 EOG channel and 9 good Stimulus channels, and 1 EEG channel is bad, which is "EEG053". Moreover, there are no bad channels in the Stimulus and EOG. After that, I checked the channel's name and  randomly picked 3 channels from EEG, EOG, and Stimulus channels, which are "EEG 001", "STI 001", and "EOG 061". 

Lastly, I would like you to answer some questions about Python & coding (please answer the questions within the Markdown).

1. Rate your experience with Python on a scale from 0 (=none) to 10(=expert)

ANSWER HERE: My experience in Python: 8. 


2. Where and how did you learn Python programming? 

ANSWER HERE: I completed my Bachelor of Science in the Computer Science department, and for that reason, I was involved in different programming courses from my Bachelor studies. Moreover, I worked in three offices where I used Python. As a master's student in the Data Science program, I always use Python to complete my tasks. Thank you.    


3. What is your experience with Github and Git?

ANSWER HERE: 8 out of 10. I use GitHub to store of my projects. 


Please insert the code where needed and send me the changed Python notebook file within a week (latest). 